# Loading Data and applying Collaborative Filtering!

In [1]:
import wikipedia as wiki
from sklearn.model_selection import train_test_split


In [2]:
import pandas as pd

Data = pd.read_csv('ratings_10000_7381_25.csv')
print(len(Data))

Data_matrix= Data.groupby(['user_id','book_id']).rating.mean().unstack(fill_value=-1)
print ("Data has", Data_matrix.shape[0], "Rows,", Data_matrix.shape[1], "Columns")

Data_matrix.head()




1466230
Data has 10000 Rows, 7381 Columns


book_id,1,2,3,4,5,6,7,8,9,10,...,9974,9976,9977,9981,9984,9985,9990,9991,9995,9999
user_id,,,,,,,,,,,,,,,,,,,,,
4,-1,5,-1,4,4,-1,4,4,-1,5,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,3,-1,-1,2,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
35,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
36,4,-1,3,5,3,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
41,5,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [3]:
train_data, test_data = train_test_split(Data, 
    train_size=0.9, random_state = 20)


C:\Users\Vansh Narula\AppData\Local\conda\conda\envs\tf_gpu\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [4]:
trainData= train_data.groupby(['user_id','book_id']).rating.mean().unstack(fill_value=-1)
print ("trainData has", trainData.shape[0], "Rows,", trainData.shape[1], "Columns")
trainData.head()

trainData has 10000 Rows, 7381 Columns


book_id,1,2,3,4,5,6,7,8,9,10,...,9974,9976,9977,9981,9984,9985,9990,9991,9995,9999
user_id,,,,,,,,,,,,,,,,,,,,,
4,-1,5,-1,4,4,-1,4,4,-1,5,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,3,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
35,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
36,4,-1,3,5,3,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
41,5,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [5]:
import numpy as np
test_UserIDs = test_data['user_id'].values
test_bookIDs = test_data['book_id'].values

test_matrix = test_data.values
test_matrix[0]

array([32070,   879,     5], dtype=int64)

In [6]:
training_matrix = trainData.values
unique_bookIDs = trainData.columns
unique_userIDs = trainData.index

print(training_matrix)
print(unique_bookIDs)
print(unique_userIDs)

#get index in training matrix from movie Id
Book_ID_Index = {}
for i in range(len(unique_bookIDs)):
    Book_ID_Index[unique_bookIDs[i]] = i

#get index in training matrix from User Id
User_ID_Index = {}
for i in range(len(unique_userIDs)):
    User_ID_Index[unique_userIDs[i]] = i
    
# training_matrix[User_ID_Index[]]

[[-1  5 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 ...
 [-1  5  4 ... -1 -1 -1]
 [-1  4 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]]
Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            9974, 9976, 9977, 9981, 9984, 9985, 9990, 9991, 9995, 9999],
           dtype='int64', name='book_id', length=7381)
Int64Index([    4,     7,    35,    36,    41,    61,    75,    89,   116,
              119,
            ...
            53332, 53337, 53352, 53364, 53366, 53372, 53373, 53381, 53403,
            53411],
           dtype='int64', name='user_id', length=10000)


In [7]:
from numpy import linalg as LA
def remove_negative(a):
    return (a!=-1)*a

def Cosine_Similarity(a, b):
    a = remove_negative(a)
    b = remove_negative(b)
    sim = a.dot(b) /((LA.norm(a,ord=2)) * (LA.norm(b,ord=2)))
#     print(sim)
    return sim

In [8]:
Mean_ratings={}
for id in unique_userIDs:
    list = training_matrix[User_ID_Index[id]]
    mean = np.average(list, weights=(list >= 0))
    Mean_ratings[id]=mean

In [9]:
Mean_ratings[test_matrix[-1][0]]

4.567164179104478

In [10]:
k=0.0015
def get_recommendation(user_Id,book_Id):
    rec = Mean_ratings[user_Id] + k* np.sum([(Cosine_Similarity(training_matrix[User_ID_Index[user_Id]],training_matrix[User_ID_Index[id2]])*(training_matrix[User_ID_Index[id2]][Book_ID_Index[book_Id]] - Mean_ratings[id2])) for id2 in unique_userIDs if training_matrix[User_ID_Index[id2]][Book_ID_Index[book_Id]] != -1 and id2!=user_Id ]) 
    return rec   

predictions_cosine=[]   
# count=0
for test_data_curr in test_matrix:
#     count+=1
#     print(count)
    rec = get_recommendation(test_data_curr[0],test_data_curr[1])
    predictions_cosine.append(rec)

In [11]:
# for x in predictions_cosine:
#     print(x)

In [12]:
# user_Id = test_matrix[-1][0]
# book_Id = test_matrix[-1][1]


# print(get_recommendation(test_matrix[-1][0],test_matrix[-1][1]))

In [13]:
# sum = 0
# for id2 in unique_userIDs:
#     if training_matrix[User_ID_Index[id2]][Book_ID_Index[book_Id]] != -1 and id2!=user_Id:
#         print('mean rating for',id2,'=', Mean_ratings[id2])
#         print('rating=',training_matrix[User_ID_Index[id2]][Book_ID_Index[book_Id]])
#         print(' Cosine Similarity=',Cosine_Similarity(training_matrix[User_ID_Index[user_Id]],training_matrix[User_ID_Index[id2]]))
#         sum+= (Cosine_Similarity(training_matrix[User_ID_Index[user_Id]],training_matrix[User_ID_Index[id2]])*(training_matrix[User_ID_Index[id2]][Book_ID_Index[book_Id]] - Mean_ratings[id2]))
#         print(sum)

In [14]:
# sum2 = 0.0015 * sum
# pred = Mean_ratings[user_Id] + sum2
# pred



In [15]:
# rmse(pred,y_true[-4])

In [16]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
y_true = test_data['rating'].values
print(y_true)
y_pred = np.asarray(predictions_cosine)
print(y_pred)

print('mean absolute error of Cosine = ' , mean_absolute_error(y_true, y_pred))

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

print('root mean square error of Cosine = ' , rmse(y_pred,y_true))


[5 3 5 ... 5 5 5]
[4.38911373 3.84625245 4.67900393 ... 4.16074895 4.17804314 4.8626987 ]
mean absolute error of Cosine =  0.6950939374158898
root mean square error of Cosine =  0.8840569277174134


# Pearson Similarity

In [17]:
from copy import copy, deepcopy
new_matrix = deepcopy(training_matrix).astype(float)

#make a new training matrix with mean subtracted values and fill empty values with 0
for id in unique_userIDs:
    list = new_matrix[User_ID_Index[id]]
    for j in range(len(list)):
        if list[j] != -1:
            new_matrix[User_ID_Index[id],j] = new_matrix[User_ID_Index[id],j] - Mean_ratings[id]
        else:
            new_matrix[User_ID_Index[id],j] = 0



In [18]:
def set_threshold(threshold):
    def pearson_similarity(user_id1,user_id2, k=threshold):
        
        user1 = new_matrix[User_ID_Index[user_id1]]
        user2 = new_matrix[User_ID_Index[user_id2]]
        indices = np.logical_and(user1!=0,user2!=0)
        a=user1[indices]
        b=user2[indices]
        if np.sum(indices)<k:
            return 0
        
        if LA.norm(a,ord=2)==0 or LA.norm(b,ord=2)==0:
            return 0
        else:
            return a.dot(b) /((LA.norm(a,ord=2)) * (LA.norm(b,ord=2)))
    return pearson_similarity  
            

In [19]:
k=0.0015
def get_recommendation_pearson(user_Id,book_id,threshold):
    score = set_threshold(threshold)
    rec = Mean_ratings[user_Id] + k* np.sum([score(user_Id,id2)*(training_matrix[User_ID_Index[id2]][Book_ID_Index[book_id]] - Mean_ratings[id2]) for id2 in unique_userIDs if id2!= user_Id and training_matrix[User_ID_Index[id2]][Book_ID_Index[book_id]] != -1]) 
    return rec   

def get_predictions_pearson(test_matrix,threshold):
    predictions_pearson=[]        
    for test_data_curr in test_matrix:
        rec = get_recommendation_pearson(test_data_curr[0],test_data_curr[1],threshold)
        predictions_pearson.append(rec)
    return np.asarray(predictions_pearson)


In [20]:
predictions_pearson = get_predictions_pearson(test_matrix,6)

In [21]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
y_true = test_data['rating'].values
print(y_true)
print(predictions_pearson)

print('mean absolute error of pearson = ' , mean_absolute_error(y_true, predictions_pearson))
print('root mean square error of pearson= ' , rmse(predictions_pearson,y_true))


[5 3 5 ... 5 5 5]
[4.39671318 3.85445347 4.673134   ... 4.16200837 4.18049035 4.70224527]
mean absolute error of pearson =  0.6825907417987471
root mean square error of pearson=  0.8715705689186595
